In [1]:
import time

import numpy as np
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

from pyepo.model.opt import optModel


class optDatasetConstrs(Dataset):
    """
    This class is Torch Dataset for optimization problems with active constraints.

    Attributes:
        model (optModel): Optimization models
        feats (np.ndarray): Data features
        costs (np.ndarray): Cost vectors
        sols (np.ndarray): Optimal solutions
        objs (np.ndarray): Optimal objective values
    """

    def __init__(self, model, feats, costs):
        """
        A method to create a optDataset from optModel

        Args:
            model (optModel): an instance of optModel
            feats (np.ndarray): data features
            costs (np.ndarray): costs of objective function
        """
        if not isinstance(model, optModel):
            raise TypeError("arg model is not an optModel")
        self.model = model
        # data
        self.feats = feats
        self.costs = costs
        # find optimal solutions
        self.sols, self.objs = self._getSols()

    def _getSols(self):
        """
        A method to get optimal solutions for all cost vectors
        """
        sols = []
        objs = []
        print("Optimizing for optDataset...")
        time.sleep(1)
        for c in tqdm(self.costs):
            try:
                sol, obj = self._solve(c)
                constrs = self._getActiveConstrs()
            except:
                raise ValueError(
                    "For optModel, the method 'solve' should return solution vector and objective value."
                )
            sols.append(sol)
            objs.append([obj])
        return np.array(sols), np.array(objs)

    def _solve(self, cost):
        """
        A method to solve optimization problem to get an optimal solution with given cost

        Args:
            cost (np.ndarray): cost of objective function

        Returns:
            tuple: optimal solution (np.ndarray) and objective value (float)
        """
        self.model.setObj(cost)
        sol, obj = self.model.solve()
        return sol, obj
    
    def _getActiveConstrs(self):
        """
        A method to get active constraints with current optimal solution

        Returns:
            np.ndarray: normal vector of constraints
        """
        for constr in self.model._model.getConstrs():
            if abs(constr.Slack) < 1e-5:
                print(constr.ConstrName, "is active.")
                print(self.model._model.getRow(constr))

    def __len__(self):
        """
        A method to get data size

        Returns:
            int: the number of optimization problems
        """
        return len(self.costs)

    def __getitem__(self, index):
        """
        A method to retrieve data

        Args:
            index (int): data index

        Returns:
            tuple: data features (torch.tensor), costs (torch.tensor), optimal solutions (torch.tensor) and objective values (torch.tensor)
        """
        return (
            torch.FloatTensor(self.feats[index]),
            torch.FloatTensor(self.costs[index]),
            torch.FloatTensor(self.sols[index]),
            torch.FloatTensor(self.objs[index]),
        )

Auto-Sklearn cannot be imported.


## Data Set and Optimization Solver 

In [2]:
import pyepo

In [3]:
# generate data
grid = (2,2) # grid size
num_data = 1000 # number of training data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

In [4]:
from pyepo.model.grb import shortestPathModel
# set solver
optmodel = shortestPathModel(grid)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
for i, e in enumerate(optmodel.arcs):
    if sol[i] > 1e-3:
        print(e)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 0.5009689961416153
(0, 2)
(2, 3)


In [5]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)
# get training and test data set
dataset_train = optDatasetConstrs(optmodel, x_train[:10], c_train[:10])

Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 2851.52it/s]

R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x[0,1] + -1.0 x[0,2]
R1 is active.
x[0,1] + -1.0 x[1,3]
R2 is active.
x[0,2] + -1.0 x[2,3]
R3 is active.
x[1,3] + x[2,3]
R0 is active.
-1.0 x